In [2]:
import pandas as pnd
import numpy as np
from scipy import stats

# Set up some global config and variables
pnd.options.mode.chained_assignment = None
np.seterr(all='ignore')

df = pnd.read_csv('jhu-daily-reports.csv')
df['Active']  = df.Confirmed - (df.Deaths + df.Recovered)
samples = df[['Date', 'Country']].groupby('Date').Country.nunique()
days = samples[samples > 1].index.tolist()
df = df[df['Date'].isin(days)]

country_level = df.groupby(['Country', 'Date'], as_index=False).sum()
def state_data(country):
    return df[df['Country'] == country].groupby(['State', 'Date'], as_index=False).sum()
def county_data(state):
    return df[(df['Country'] == 'US') & (df['State'] == state)].groupby(['County', 'Date'], as_index=False).sum()

root = state_data('US').sort_values('Date')
for date in root.Date.unique()[40:]:
    for state in root.State.unique():
        values = root[(root.State == state) & (root.Date <= date)].sort_values('Date').tail(21)[['Confirmed', 'Confirmed_New']]
        if values.empty:
            continue
        slope, intercept, r_value, p_value, std_err = stats.linregress(values.Confirmed, values.Confirmed_New)
        root.loc[(root.State == state) & (root.Date == date), 'Slope'] = slope
slope = root[root.Date >= '03-01-2020']
slope = slope[['State', 'Date', 'Slope']]
slope.fillna(0, inplace=True)

In [31]:
import geopandas as gpd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from datetime import datetime

usa = gpd.read_file('./maps/states.shp')[1:50]
slope = slope[slope.State.isin(usa.STATE_ABBR)]
merged = usa.set_index('STATE_ABBR').join(slope.set_index('State'))

norm = colors.TwoSlopeNorm(vmin=-0.2, vcenter=0., vmax=0.6)

for date in merged.Date.unique():
    fig, ax = plt.subplots(1, figsize=(6, 2)) #figsize=(18, 6))
    usa.plot(ax=ax, color='lightgrey')
    merged[merged.Date == date].plot(
        column='Slope',
        cmap='RdYlGn_r',
        norm=norm,
        ax=ax,
        legend=True,
    )
    ax.annotate(date, xy=(50, 20), xycoords='figure pixels', size='7')
    ax.axis('off')
    plt.savefig('./output/slope-map-%s.png' % datetime.strptime(date, '%m-%d-%Y').strftime('%m-%d'), dpi=300, bbox_inches='tight')
    plt.close()

In [30]:
print(merged[merged.Date=='06-17-2020'].Slope.min())
print(merged[merged.Date=='05-17-2020'].Slope.max())

-0.06824271278344404
0.05768443181631546
